### PyNAMod
1) Получение огрубленных моделей структур ДНК, генерация линейных ДНК, объединение моделей.
2) Исследование конформаций с помощью моделирования методом Монте Карло.

![alt text](params.png "Title")

### Установка
https://github.com/intbio/PyNAMod \
Создание окружения:\
conda env create --file environment.yml

### Анализ полноатомоных моделей

In [1]:
import pynamod
import torch

/home/v_vasilev/.conda/envs/pynamod_env/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/v_vasilev/.conda/envs/pynamod_env/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


/home/v_vasilev/.conda/envs/pynamod_env/lib/python3.11/site-packages/Bio/Application/__init__.py:40: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(
/home/v_vasilev/.conda/envs/pynamod_env/lib/python3.11/site-packages/MDAnalysis/topology/PDBParser.py:331: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
# Инициализация с помощью объекта mda Universe, pdb файла или pdb id
nucl = pynamod.CG_Structure(pdb_id='3LZ0')

Sending GET request to https://files.rcsb.org/download/3LZ0.pdb to fetch 3LZ0's pdb file as a string.


Получение геометрических параметров

In [17]:
nucl.analyze_dna(leading_strands=['I'])

Получение огрубленной модели белка

In [18]:
nucl.analyze_protein()

Визуализация

In [6]:
# Перемещение первой пары в начало координат
nucl.dna.move_to_coord_center()

nucl.view_structure()

NGLWidget()

### Генерация линейной ДНК

In [19]:
seq = 'atcg'*2
dna_gen = pynamod.CG_Structure()
dna_gen.build_dna(seq)

In [8]:
dna_gen.view_structure()


NGLWidget()

In [30]:
dna_gen.h

AttributeError: 'CG_Structure' object has no attribute 'h'

### Объединение структур

Инициализация стурктуры с белком cas9

In [24]:
cas = pynamod.CG_Structure(pdb_id='5Y36')
cas.analyze_dna(leading_strands=['C'],sel='segid C D')

Sending GET request to https://files.rcsb.org/download/5Y36.pdb to fetch 5Y36's pdb file as a string.


In [25]:
cas.analyze_protein()

In [26]:
cas.view_structure()

NGLWidget()

Объединение с нуклеосомой и линейной ДНК

In [22]:
nucl.append_structures([dna_gen,cas])

In [28]:
nucl.dna.step_params[10] = torch.ones(6)*100

In [29]:
nucl.view_structure()

NGLWidget()

### Моделирование методом Монте-Карло

Подготовка расчетов энергий

In [58]:
import torch
import numpy as np

In [63]:
en = pynamod.Energy()

# Инициализация матриц силовых констант
nucl.move_to_torch()
en.set_energy_matrices(nucl)

Полная энергия: энергия изгиба ДНК + энергия электростатических взаимодействий + энергия взаимодействия частиц (в виде потенциала Леннарда Джонса или сигмоиды)

In [64]:
en.get_full_energy(nucl)

tensor(10760.6611)

Генерация списка шагов, которые будут изменяться при моделирование

In [66]:
movable_steps = torch.ones(nucl.steps_params.shape[0],dtype=bool)

In [67]:
movable_steps

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True])

Запуск моделирования

In [68]:
it = pynamod.Iterator(nucl,en)

In [69]:
frames, energies = it.run(target_accepted_steps=500,movable_steps=movable_steps)

Steps:   0%|          | 0/1000000.0 [00:00<?, ?it/s]

Acceptance rate:   0%|          

target accepted steps reached
accepted_steps 500


In [ ]:
frames

### Применение ограничений на взаимное положение пар ДНК

Для получения определенных конформаций структуры в расчет можно вводить ожидаемые геометрические параметры между любыми двумя парами ДНК

In [20]:
from pynamod.external_forces.restraint import Restraint
from pynamod.energy.energy_constants import BDNA_step
from pynamod.geometry.bp_step_geometry import rebuild_by_full_par_frame_numba

Получение кольцевой ДНК из линейной

In [71]:
# Индексы пар, на которые накладывается ограничение
ind1,ind2 = 0, -1

# Коэффицент этого ограничения в формуле полной энергии
force_const = 500

# Целевые геометрические параметры
params = torch.Tensor(BDNA_step[6:])

# Допустимое отклонение
dev = torch.ones(1)*0.1

# Инициализация
restr = Restraint(ind1,ind2,force_const,params,dev)

Подготовка расчета энергии

In [72]:
en_linear = pynamod.Energy()
dna_gen.move_to_torch()
en_linear.set_energy_matrices(dna_gen)
en_linear.add_restraints([restr])

In [49]:
it = pynamod.Iterator(dna_gen,en_linear)
movable_steps = torch.ones(dna_gen.steps_params.shape[0],dtype=bool)

In [50]:
frames,energies = it.run(movable_steps=movable_steps,target_accepted_steps=100000)

Steps:   0%|          | 0/1000000.0 [00:00<?, ?it/s]

Acceptance rate:   0%|          

interrupted!
accepted_steps 27295


In [51]:
dna_gen.base_ref_frames = rebuild_by_full_par_frame_numba(np.array(frames[-1]))

In [52]:
dna_gen.view_structure()

NGLWidget()